In [ ]:
import geopandas as gpd
from shapely.geometry import mapping
import json
import ee

# Provide the path to your shapefile
shapefile_path = '/content/drive/MyDrive/Urmia/Boundry/boundry.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Ensure the shapefile is in the correct projection (WGS84)
gdf = gdf.to_crs(epsg=4326)

# Convert the GeoDataFrame to a GeoJSON dictionary
geojson_dict = json.loads(gdf.to_json())

# Extract the geometry coordinates
polygon_coords = geojson_dict['features'][0]['geometry']['coordinates']

# Remove the Z coordinate (altitude) from the coordinates if present
def remove_z(coords):
    return [[coord[:2] for coord in poly] for poly in coords]

polygon_coords_2d = remove_z(polygon_coords)

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-hadikheiri78')

# Create an Earth Engine Polygon object from the coordinates
try:
    roi = ee.Geometry.Polygon(polygon_coords_2d)
    print("Geometry successfully created!")
except Exception as e:
    print("Error creating geometry:", e)

# Load Sentinel-2 data for 2017 using the updated collection
s2_2017 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2017-07-25', '2017-08-31') \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B2', 'B3', 'B4', 'B8'])

# Load Sentinel-2 data for 2024 using the updated collection
s2_2024 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2024-07-25', '2024-08-18') \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B2', 'B3', 'B4', 'B8'])

# Median composite for 2017
composite_2017 = s2_2017.median().clip(roi)

# Median composite for 2024
composite_2024 = s2_2024.median().clip(roi)

# Define export parameters with 30-meter resolution
export_params = {
    'scale': 30,  # Change resolution to 30 meters
    'region': roi,
    'driveFolder': 'EarthEngine',
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13  # Ensure large enough for big exports
}

# Export 2017 composite
export_task_2017 = ee.batch.Export.image.toDrive(composite_2017, '2017_LakeU_Sentinel2', **export_params)
export_task_2017.start()

# Export 2024 composite
export_task_2024 = ee.batch.Export.image.toDrive(composite_2024, '2024_LakeU_Sentinel2', **export_params)
export_task_2024.start()

print('Exporting images to Google Drive...')


Geometry successfully created!
Exporting images to Google Drive...
